# 📍 Análisis Geoespacial de Negocios en Madrid

Este notebook presenta un análisis geoespacial interactivo de la distribución de negocios en la ciudad de Madrid. Utiliza datos categorizados y shapefiles oficiales para detectar patrones de densidad y realizar visualizaciones sobre un mapa.

---

### 🎯 **Objetivos del análisis**

1. Visualizar la distribución de negocios por barrio en Madrid.
2. Identificar áreas de alta densidad mediante clustering espacial.
3. Clasificar negocios en categorías para análisis sectorial.

---

### 🗂️ **Datos utilizados**

- `locales_todos.csv`: Información de negocios (nombre, latitud, longitud, etc.)
- `BARRIOS.shp`: Shapefile con los límites geográficos de los barrios de Madrid.

---

### 📌 **Resultados esperados**

- Mapas interactivos con folium.
- Identificación de zonas con concentración de actividad comercial.
- Dataset etiquetado útil para modelos de predicción o recomendación.

## 📂 Configuración del Entorno

En esta sección se preparan los directorios y archivos necesarios para el correcto funcionamiento del notebook. Esto incluye:

- Identificación del directorio raíz del proyecto.
- Creación de carpetas si no existen.
- Definición de rutas a archivos de entrada y salida.

> Esto garantiza la portabilidad y robustez del notebook en diferentes entornos.

In [ ]:
from pathlib import Path
import os

# Detectar la raíz del proyecto
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

# Directorios clave
data_dir = project_root / "data"
barrios_dir = data_dir / "Barrios"
outputs_dir = project_root / "outputs"

# Crear carpetas si no existen
data_dir.mkdir(parents=True, exist_ok=True)
barrios_dir.mkdir(parents=True, exist_ok=True)
outputs_dir.mkdir(parents=True, exist_ok=True)

# Rutas de archivo
locales_csv = data_dir / "locales_todos.csv"
procesado_csv = data_dir / "MAPS_locales_procesado.csv"
categorizado_csv = data_dir / "MAPS_locales_categorizados.csv"
shapefile_path = barrios_dir / "BARRIOS.shp"
mapa_html = outputs_dir / "mapa_locales.html"

# Mostrar confirmación
print(f"📁 Proyecto raíz: {project_root}")
print(f"📄 locales_todos.csv → {locales_csv}")
print(f"📄 BARRIOS.shp → {shapefile_path}")
print(f"💾 Salida HTML → {mapa_html}")

## 🔑 Configuración de la clave API de Google

Se carga la clave API de Google almacenada en un archivo `.env`. Esta clave es necesaria para realizar consultas a los servicios de geolocalización (`Geocoding`) y búsqueda de lugares (`Places API`) de Google.

> ⚠️ Asegúrate de que el archivo `.env` contenga una clave válida con permisos adecuados.

In [ ]:
import os
from dotenv import load_dotenv

# Carga clave desde .env o asignación directa temporal
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

## ⚙️ Definición de Funciones Auxiliares

Aquí se definen funciones reutilizables para:

- Obtener coordenadas a partir de códigos postales.
- Consultar negocios cercanos con Google Places.
- Limpiar y transformar los resultados en un DataFrame.

Estas funciones permiten modularizar el código y facilitar su reutilización.

In [ ]:
import requests
import pandas as pd
import time

def get_coordinates(postal_code):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {'address': f'{postal_code}, Madrid, Spain', 'key': API_KEY}
    response = requests.get(url, params=params).json()
    
    if response['status'] != 'OK':
        raise ValueError(f"Error al obtener coordenadas para {postal_code}: {response['status']}")
    
    result = response['results'][0]
    
    # Validar que el país devuelto sea España mediante short_name = 'ES'
    components = result['address_components']
    country_codes = [c['short_name'] for c in components if 'country' in c['types']]
    if 'ES' not in country_codes:
        raise ValueError(f"Código postal {postal_code} no está en España (devuelto: {country_codes})")
    
    location = result['geometry']['location']
    return location['lat'], location['lng']

def get_places(lat, lng, radius=2000, business_type=None):
    places = []
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{lat},{lng}',
        'radius': radius,
        'key': API_KEY,
        'type': business_type
    }

    while True:
        res = requests.get(url, params=params).json()
        places.extend(res.get('results', []))
        if 'next_page_token' in res:
            time.sleep(2)
            params['pagetoken'] = res['next_page_token']
        else:
            break

    return places

def clean_data(places, postal_code):
    data = []
    for p in places:
        location = p.get('geometry', {}).get('location', {})
        data.append({
            'nombre': p.get('name'),
            'direccion': p.get('vicinity'),
            'codigo_postal': postal_code,
            'puntuacion_media': p.get('rating'),
            'numero_reviews': p.get('user_ratings_total'),
            'tipo_negocio': ', '.join(p.get('types', [])),
            'latitud': location.get('lat'),
            'longitud': location.get('lng')
        })
    return pd.DataFrame(data)

## 🧭 Extracción de Negocios por Código Postal

Se recorre una lista de códigos postales de Madrid y, para cada uno, se:

1. Obtienen las coordenadas del centro del área.
2. Realiza una consulta a Google Places para negocios cercanos.
3. Se transforman los resultados en un DataFrame limpio.
4. Se concatena toda la información en un solo dataset.

> Este proceso puede tardar varios minutos, dependiendo del número de códigos postales y los resultados devueltos por la API.

In [ ]:
from pathlib import Path
import time
import pandas as pd

codigos_postales = ['28001', '28002', '28003', '28004', '28005', '28006', '28007', '28008', '28009',
                    '28010', '28011', '28012', '28013', '28014', '28015', '28016', '28017', '28018', '28019',
                    '28020', '28021', '28022', '28023', '28024', '28025', '28026', '28027', '28028', '28029',
                    '28030', '28031', '28032', '28033', '28034', '28035', '28036', '28037', '28038', '28039',
                    '28040', '28041', '28042', '28043', '28044', '28045', '28046', '28047', '28048', '28049',
                    '28050', '28051', '28052', '28053', '28054', '28055'
                    ]
tipo_negocio = None  # Puedes especificar un tipo como 'restaurant', 'store', etc.

todos_los_locales = pd.DataFrame()

for cp in codigos_postales:
    try:
        lat, lng = get_coordinates(cp)
        lugares = get_places(lat, lng, business_type=tipo_negocio)
        df = clean_data(lugares, cp)
        todos_los_locales = pd.concat([todos_los_locales, df], ignore_index=True)
        print(f"✅ {len(df)} locales encontrados en {cp}")
        time.sleep(1)
    except ValueError as e:
        print(f"⚠️ Error de validación con código postal {cp}: {e}")
    except Exception as e:
        print(f"⚠️ Error inesperado con código postal {cp}: {e}")


from pathlib import Path
import os

# Subir al directorio raíz del proyecto desde notebooks/
project_root = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path.cwd().parent
data_dir = project_root / "data"
data_dir.mkdir(exist_ok=True)

# Definir nombre y guardar CSV
nombre_tipo = tipo_negocio if tipo_negocio else "todos"
output_file = data_dir / f"locales_{nombre_tipo}.csv"
todos_los_locales.to_csv(output_file, index=False)

print(f"\n✅ Archivo '{output_file.name}' generado exitosamente en '{output_file.parent}'")


## 🌍 Visualización Interactiva de Negocios

Con ayuda de **Folium**, se genera un mapa interactivo con los siguientes elementos:

- Marcadores con información detallada del negocio.
- Un **MiniMapa** que facilita la navegación.
- Estilo de mapa personalizado usando `CartoDB.VoyagerLabels`.

Cada negocio se representa con un icono personalizado, y al hacer clic se despliega un resumen con nombre, dirección, puntuación y tipo.

In [ ]:
import folium.plugins as plugins
import folium

df = pd.read_csv(locales_csv)
df = df.dropna(subset=['latitud', 'longitud'])

mapa = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()], 
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'  # Estilos --> https://leaflet-extras.github.io/leaflet-providers/preview/
    )

# Añadir MiniMap
plugins.MiniMap().add_to(mapa)

for _, row in df.iterrows():
    popup_text = f"""
    <b>{row['nombre']}</b><br>
    Dirección: {row['direccion']}<br>
    Rating: {row['puntuacion_media']} ({row['numero_reviews']} reseñas)<br>
    Tipo: {row['tipo_negocio']}
    """
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        icon=plugins.BeautifyIcon(
                     icon="home",
                     icon_shape="circle",
                     border_color='green',
                     text_color="green",
                     background_color='white'
                 ),
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=row['nombre']
    ).add_to(mapa)

mapa.save(mapa_html)
mapa

## 🏷️ Categorización de Tipos de Negocios

En esta sección se clasifica cada negocio en una categoría general a partir del contenido de su tipo (`tipo_negocio`). Esto permite agrupar los negocios en sectores más amplios y útiles para el análisis.

Se siguen los siguientes pasos:

1. **Cargar el CSV original** con los datos de los negocios.
2. **Definir una función de categorización** que analiza palabras clave dentro del tipo de negocio.
3. **Aplicar la función** a cada fila del dataset para generar una nueva columna `categoria_negocio`.
4. **Guardar el resultado** en un nuevo archivo CSV.

In [ ]:
import pandas as pd

df = pd.read_csv(locales_csv)

def categorizar_negocio(tipo_str):
    tipo_str = str(tipo_str).lower()
    if any(t in tipo_str for t in ['restaurant', 'food', 'bar', 'cafe', 'bakery', 'meal_takeaway']):
        return 'Restauración'
    elif any(t in tipo_str for t in ['lodging', 'hotel', 'hostel', 'guest_house']):
        return 'Alojamiento'
    elif any(t in tipo_str for t in ['church', 'place_of_worship', 'mosque', 'synagogue', 'hindu_temple']):
        return 'Religión / Culto'
    elif any(t in tipo_str for t in ['locality', 'political', 'city_hall', 'post_office', 'courthouse', 'embassy', 'local_government_office']):
        return 'Administración / Localidad'
    elif any(t in tipo_str for t in ['store', 'shopping_mall', 'hair_care', 'bank', 'car_repair', 'laundry', 'gym', 'pharmacy', 'doctor', 'real_estate_agency']):
        return 'Comercio / Servicios'
    elif any(t in tipo_str for t in ['school', 'university', 'secondary_school']):
        return 'Educación'
    elif any(t in tipo_str for t in ['health']):
        return 'Salud'
    elif any(t in tipo_str for t in ['finance', 'insurance_agency']):
        return 'Finanzas y Seguros'
    elif any(t in tipo_str for t in ['general_contractor']):
        return 'Construcción / Reformas'
    elif any(t in tipo_str for t in ['park']):
        return 'Naturaleza / Recreación'
    elif any(t in tipo_str for t in ['tourist_attraction']):
        return 'Turismo / Atracción'
    else:
        return 'Otros / No categorizado'

df['categoria_negocio'] = df['tipo_negocio'].apply(categorizar_negocio)
df.to_csv(categorizado_csv, index=False)

## 🔍 Refinamiento de la Categorización mediante el Nombre del Negocio

Después de realizar una categorización inicial basada en el tipo (`tipo_negocio`), muchos registros aún quedan clasificados como "Otros / No categorizado". Para mejorar la calidad del etiquetado, se realiza una segunda pasada utilizando el contenido del campo `nombre`.

Esta estrategia permite detectar patrones frecuentes en los nombres (como "bar", "peluquería", "taller", etc.) y reclasificar adecuadamente esos negocios en categorías más precisas.

Este refinamiento puede mejorar hasta un 20% de los casos inicialmente no categorizados, incrementando así la utilidad del análisis posterior.

In [ ]:
def clasificar_por_nombre(nombre, tipo_actual):
    nombre = str(nombre).lower()
    if tipo_actual == 'Otros / No categorizado':
        if 'bar' in nombre: return 'Restauración'
        if 'peluquería' in nombre or 'hair' in nombre: return 'Comercio / Servicios'
        if 'taller' in nombre or 'auto' in nombre: return 'Comercio / Servicios'
        # ... y así puedes mejorar hasta un 20% más de casos
    return tipo_actual

df['categoria_negocio'] = df.apply(lambda row: clasificar_por_nombre(row['nombre'], row['categoria_negocio']), axis=1)

## 🧹 Filtrado de Negocios según Relevancia

Para asegurar que el análisis se base en datos representativos y confiables, se aplican dos criterios de limpieza:

- Se eliminan aquellos negocios que no tienen puntuación media (`NaN`).
- Se excluyen los negocios con menos

In [ ]:
df = df[df['puntuacion_media'].notna() & (df['numero_reviews'] >= 3)]

In [ ]:
df.to_csv(procesado_csv, index=False)

## 🚀 Feature engineering avanzado

En esta sección enriquecemos el dataset con tres nuevas columnas que capturan señal geoespacial:

1. **Distancia al centro de la ciudad**  
2. **Densidad de locales en un radio de 1 km**  
3. **Clusterización espacial (DBSCAN)**

In [ ]:
# --- Densidades a varios radios ---
from sklearn.neighbors import BallTree
import numpy as np

coords_rad = np.deg2rad(df[['latitud','longitud']].values)
tree = BallTree(coords_rad, metric='haversine')

# Radios en km
for r_km in [0.5, 1.0, 2.0]:
    col = f'density_{int(r_km*1000)}m'
    df[col] = tree.query_radius(
        coords_rad,
        r=r_km / 6371.0,  # convierte km → radianes
        count_only=True
    )

# Ratio competencia cercana vs área más amplia
df['ratio_500m_2km'] = df['density_500m'] / (df['density_2000m'] + 1)

df[['density_500m','density_1000m','density_2000m','ratio_500m_2km']].head()

In [ ]:
# --- Transformaciones polinómicas e interacciones ---
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

# Seleccionamos dos features numéricas para expandir
poly_feats = ['dist_city_center_km', 'density_1000m']
poly = PolynomialFeatures(degree=2, include_bias=False)

# Generar nuevas features
arr = poly.fit_transform(df[poly_feats])
names = poly.get_feature_names_out(poly_feats)
poly_df = pd.DataFrame(arr, columns=names, index=df.index)

# Concatenar con el DataFrame principal
df = pd.concat([df, poly_df], axis=1)

# Ejemplo de nuevas columnas
df[names].head()

In [ ]:
# --- Feature Engineering Avanzado ---
from haversine import haversine
from sklearn.neighbors import BallTree
from sklearn.cluster import DBSCAN
import numpy as np

# Definir centro de ciudad (ejemplo: Madrid centro)
city_center = (40.4168, -3.7038)

# 1) Distancia al centro en km
df['dist_city_center_km'] = df.apply(
    lambda r: haversine((r.latitud, r.longitud), city_center),
    axis=1
)

# 2) Densidad de locales en un radio de 1 km
coords_rad = np.deg2rad(df[['latitud','longitud']].values)
tree = BallTree(coords_rad, metric='haversine')
radius = 1/6371.0  # radio de 1 km en radianes
df['local_density_1km'] = tree.query_radius(coords_rad, r=radius, count_only=True)

# 3) Clusterización geográfica con DBSCAN
db = DBSCAN(eps=0.5/6371.0, min_samples=10, metric='haversine')
df['cluster_zone'] = db.fit_predict(coords_rad).astype(str)

# Revisión de las nuevas columnas
df[['dist_city_center_km','local_density_1km','cluster_zone']].head()

### 📊 Comparativa Antes y Después del Filtrado

Para entender el impacto del filtrado, es útil comparar la cantidad de registros antes y después de aplicar los criterios de calidad.

```python
total_antes = len(df_original)
total_despues = len(df)
print(f"Número de negocios antes del filtrado: {total_antes}")
print(f"Número de negocios después del filtrado: {total_despues}")
print(f"🔻 Registros eliminados: {total_antes - total_despues}")

## ⭐ Cálculo de Valoración Ajustada

Para obtener una medida más realista de la calidad de cada negocio, se calcula una **valoración ajustada** que tiene en cuenta tanto la puntuación media como el número de reseñas.

La fórmula utilizada penaliza a los negocios con pocas valoraciones, reduciendo la confianza en sus puntuaciones altas o bajas. Así se evita sobrevalorar negocios con una única review excelente, por ejemplo.

La fórmula aplicada es:

In [ ]:
import numpy as np

df['valoracion'] = df.apply(
    lambda row: row['puntuacion_media'] * (1 - np.exp(-row['numero_reviews'] / 10)), axis=1
)

## ⚖️ Normalización de la Valoración por Categoría

Para poder comparar de forma justa las valoraciones entre negocios de una misma categoría, se realiza una **normalización Min-Max** dentro de cada grupo de `categoria_negocio`.

Esto escala los valores de la columna `valoracion` al rango [0, 1], pero de forma **independiente por categoría**, conservando la proporcionalidad interna y permitiendo comparaciones relativas dentro de cada grupo.

> 📈 Esta normalización es útil para visualizaciones comparativas y para entrenar modelos que se benefician de variables en un mismo rango.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df['valoracion_norm'] = df.groupby('categoria_negocio')['valoracion'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='categoria_negocio', y='valoracion_norm')
plt.xticks(rotation=45, ha='right')
plt.title('Distribución de Valoraciones Normalizadas por Categoría')
plt.xlabel('Categoría de Negocio')
plt.ylabel('Valoración Normalizada')
plt.tight_layout()
plt.show()

## 🗺️ Mapa Interactivo de Negocios con Valoración

Se genera un mapa interactivo con `folium` para visualizar geográficamente la distribución de los negocios, codificados por color según su **valoración ajustada**.

### Características del mapa:

- 🟢 **Color**: El color de cada punto indica la valoración (escala de rojo a verde).
- 🔍 **MiniMap**: Se incluye una vista general secundaria en la esquina inferior derecha para facilitar la navegación.
- 📍 **Marcadores**: Cada negocio aparece como un punto circular con información emergente (`popup`) mostrando su nombre y valoración.
- 🎨 **Leyenda de color**: Se añade un colormap con la escala de valores (1 a 5) como referencia visual.

> 🧭 Esta visualización permite identificar zonas con mayor o menor concentración de negocios bien valorados y puede ayudar en análisis espaciales o decisiones estratégicas.

In [ ]:
import folium
from folium import plugins
import branca.colormap as cm

colormap = cm.LinearColormap(['red', 'orange', 'green'], vmin=1, vmax=5)
colormap.caption = 'Valoración de los negocios'

mapa = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()], 
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'  # Estilos --> https://leaflet-extras.github.io/leaflet-providers/preview/
    )


# Añadir MiniMap
plugins.MiniMap().add_to(mapa)

# Añadir marcadores como nube de puntos
for _, row in df.iterrows():
    color = colormap(row['valoracion'])
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"{row['nombre']} ({row['valoracion']:.1f})"
    ).add_to(mapa)

colormap.add_to(mapa)

mapa

## 🔥 Mapa de Calor de Densidad de Negocios

Se crea un **mapa de calor interactivo** para visualizar la densidad de negocios dentro del área analizada. Esta visualización permite identificar zonas con mayor concentración de actividad comercial.

### Elementos clave del mapa:

- 🌍 **Mapa base** centrado en el promedio de coordenadas del dataset.
- 🔎 **MiniMap** auxiliar en la esquina inferior derecha para facilitar el desplazamiento.
- 🔥 **Mapa de calor (HeatMap)** generado a partir de las coordenadas geográficas (`latitud`, `longitud`) de cada negocio.
- 🎨 **Leyenda de densidad** que representa la concentración de negocios desde zonas poco densas (azul claro) hasta zonas muy densas (rojo).

> 📌 Esta herramienta resulta útil para análisis territoriales, estudios de competencia o identificación de oportunidades de localización comercial.

In [ ]:
import folium
from folium.plugins import HeatMap, MiniMap
import branca.colormap as cm

# Crear el mapa base
mapa_densidad = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# Añadir el MiniMap
MiniMap().add_to(mapa_densidad)

# Preparar datos para el mapa de calor (sin valoración)
heat_data = [[row['latitud'], row['longitud']] for index, row in df.iterrows()]

# Añadir el mapa de calor
HeatMap(
    heat_data,
    radius=15,
    blur=12,
    min_opacity=0.3,
    max_zoom=1
).add_to(mapa_densidad)

# Crear una leyenda simple de densidad
colormap = cm.LinearColormap(
    colors=['lightblue', 'orange', 'red'],
    index=[0, 0.5, 1],
    vmin=0,
    vmax=1
)
colormap.caption = 'Densidad de negocios (baja a alta)'
colormap.add_to(mapa_densidad)

# Mostrar el mapa
mapa_densidad

## 🌡️ Mapa de Calor Ponderado por Valoración Ajustada

En esta versión del mapa de calor, cada punto no solo aporta su ubicación, sino también una **ponderación según la valoración ajustada** del negocio. Esto permite visualizar no solo dónde hay más negocios, sino dónde se concentran los **mejor valorados**.

### Detalles:

- Los puntos se ponderan utilizando la columna `valoracion`, lo que realza visualmente las áreas con mejor reputación.
- El mapa muestra tanto densidad como calidad percibida, facilitando decisiones estratégicas más informadas.

In [ ]:
# Nuevo mapa base para el mapa ponderado
mapa_valorado = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

MiniMap().add_to(mapa_valorado)

# Mapa de calor ponderado por valoración
heat_val = [[row['latitud'], row['longitud'], row['valoracion']] for _, row in df.iterrows()]

HeatMap(
    heat_val,
    radius=15,
    blur=12,
    min_opacity=0.3,
    max_zoom=1,
    max_val=5
).add_to(mapa_valorado)

mapa_valorado

## 🗂️ Mapa Interactivo por Categoría de Negocio

En esta sección se genera un **mapa interactivo** que representa los negocios agrupados por su categoría. Cada punto se colorea de forma única en función del grupo al que pertenece, permitiendo una lectura rápida de la distribución sectorial.

### Etapas del proceso:

1. 📥 **Carga del dataset** previamente categorizado (`categorizado_csv`).
2. 🧹 **Filtrado de registros sin categoría válida** (`NaN`).
3. 📊 **Conteo de negocios por categoría** para su uso en la leyenda.
4. 🗺️ **Generación del mapa base** centrado en la zona media del dataset.
5. 🎨 **Asignación de colores únicos** a cada categoría mediante `matplotlib.cm.tab20`.
6. 📍 **Marcadores individuales** para cada negocio con `popup` informativo.
7. 🧾 **Leyenda personalizada** insertada como HTML flotante, mostrando color, nombre y cantidad por categoría.
8. 💾 **Guardado del resultado** en un archivo HTML para su posterior visualización o despliegue.

> 🧭 Esta visualización resulta ideal para explorar cómo se distribuyen espacialmente los distintos sectores económicos y analizar posibles patrones de concentración por tipo de actividad.

In [ ]:
import pandas as pd
import folium
import matplotlib.cm as cm
from folium.plugins import MiniMap
from folium import Element

# 1. Cargar el archivo con categorías
df = pd.read_csv(categorizado_csv)

# 2. Eliminar registros sin categoría válida
df = df.dropna(subset=['categoria_negocio'])

# 3. Agrupar por categoría
categoria_info = (
    df.groupby('categoria_negocio')
    .agg(n_negocios=('nombre', 'count'))
    .to_dict(orient='index')
)

# 4. Crear mapa base
mapa_categorias = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=13,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# Añadir MiniMap
MiniMap().add_to(mapa_categorias)

# 5. Asignar colores únicos por categoría
categorias = sorted(df['categoria_negocio'].unique())
colormap = cm.get_cmap('tab20', len(categorias))

def get_color_categoria(cat):
    idx = categorias.index(cat)
    rgba = colormap(idx)
    rgb = tuple(int(255 * c) for c in rgba[:3])
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

# 6. Añadir marcadores
for _, row in df.iterrows():
    color = get_color_categoria(row['categoria_negocio'])
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"{row['nombre']}<br>{row['categoria_negocio']}"
    ).add_to(mapa_categorias)

# 7. Leyenda visual por categoría
legend_html = f"""
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 280px;
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:13px;
    padding: 10px;
    overflow-y: auto;
    max-height: 300px;
">
<b>Tipos de negocio</b><br>
{''.join([
    f"<div style='margin-top:4px;'><span style='background-color:{get_color_categoria(cat)};width:12px;height:12px;display:inline-block;margin-right:6px;'></span>{cat} ({categoria_info[cat]['n_negocios']})</div>"
    for cat in categorias
])}
</div>
"""

mapa_categorias.get_root().html.add_child(Element(legend_html))

# 8. Guardar HTML
mapa_categoria_html = outputs_dir / "mapa_categorias.html"
mapa_categorias.save(mapa_categoria_html)
print(f"✅ Mapa por tipo de negocio guardado en: {mapa_categoria_html}")

## 🧭 Mapa de Densidad de Negocios por Barrio (Shapefile)

Se genera un mapa coroplético basado en un shapefile de barrios de Madrid, que representa la **densidad relativa de negocios por zona**. Este análisis espacial permite detectar patrones geográficos y desequilibrios en la distribución comercial.

### Fases del proceso:

1. 🗂️ **Carga del shapefile** con geometrías de los barrios (`shapefile_path`).
2. 📍 **Conversión de coordenadas** del dataframe `df` a un `GeoDataFrame`.
3. 🔄 **Unificación de sistemas de referencia (CRS)** entre capas geoespaciales.
4. 🧩 **Unión espacial (spatial join)** para asignar cada negocio a su barrio.
5. 📊 **Conteo de negocios por barrio** y fusión con el shapefile.
6. 📈 **Normalización** de la cantidad de negocios para representar una escala relativa de densidad.
7. 🗺️ **Creación del mapa base** con `folium` y estilos personalizados.
8. 🌈 **Leyenda de colores** que representa la densidad relativa (de azul claro a rojo).
9. 🧱 **Dibujo de polígonos** con `GeoJson`, mostrando nombre del barrio y número de negocios.
10. 💾 **Exportación del mapa** como archivo HTML interactivo.

> 📌 Este tipo de visualización permite realizar análisis territoriales precisos y detectar barrios con mayor o menor concentración de actividad comercial.

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MiniMap
import branca.colormap as cm
from shapely.geometry import Point

# Función auxiliar para asegurar CRS coincidente
def ensure_same_crs(gdf1, gdf2):
    """
    Alinea el CRS de gdf2 al de gdf1 si son distintos.
    Devuelve una copia reproyectada de gdf2 si es necesario.
    """
    if gdf1.crs != gdf2.crs:
        print(f"⚠️ CRS diferente detectado: {gdf1.crs} ≠ {gdf2.crs}")
        print("🔄 Reproyectando gdf2 al CRS de gdf1...")
        return gdf2.to_crs(gdf1.crs)
    return gdf2

# 1. Cargar el shapefile de barrios de Madrid (ruta segura desde variable shapefile_path)
gdf_barrios = gpd.read_file(shapefile_path)

# 2. Convertir DataFrame df a GeoDataFrame con puntos
gdf_negocios = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitud, df.latitud), crs='EPSG:4326')

# 3. Reproyectar barrios si tienen otro CRS
gdf_barrios = ensure_same_crs(gdf_negocios, gdf_barrios)

# 4. Unión espacial: asignar barrio a cada negocio
gdf_completo = gpd.sjoin(gdf_negocios, gdf_barrios, how='left', predicate='within')

# 5. Contar negocios por barrio
conteo = gdf_completo.groupby('NOMBRE').size().reset_index(name='num_negocios')
gdf_barrios = gdf_barrios.merge(conteo, on='NOMBRE', how='left')
gdf_barrios['num_negocios'] = gdf_barrios['num_negocios'].fillna(0)

# 6. Normalizar densidad
gdf_barrios['densidad_norm'] = (
    gdf_barrios['num_negocios'] - gdf_barrios['num_negocios'].min()
) / (
    gdf_barrios['num_negocios'].max() - gdf_barrios['num_negocios'].min()
)

# 7. Crear el mapa base
mapa_barrios = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=12,
    tiles='CartoDB.VoyagerLabelsUnder'
)

# 8. Añadir MiniMap
MiniMap().add_to(mapa_barrios)

# 9. Crear leyenda
colormap = cm.LinearColormap(['lightblue', 'orange', 'red'], vmin=0, vmax=1)
colormap.caption = 'Densidad normalizada de negocios por barrio'
colormap.add_to(mapa_barrios)

# 10. Dibujar los polígonos
folium.GeoJson(
    gdf_barrios,
    name="Barrios por densidad",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['densidad_norm']),
        'color': 'black',
        'weight': 0.7,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOMBRE", "num_negocios"],
        aliases=["Barrio", "Nº negocios"],
        localize=True
    )
).add_to(mapa_barrios)

# 11. Guardar el mapa de barrios en HTML
mapa_barrios_html = outputs_dir / "mapa_barrios.html"
mapa_barrios.save(mapa_barrios_html)

# 12. Mostrar el mapa interactivo
mapa_barrios


## 🗺️ Integración de las predicciones en el mapa

Cargamos el clasificador entrenado, predecimos la clase y probabilidad para cada local, 
y los representamos en un mapa interactivo con colores según la predicción.

In [ ]:
import folium
import joblib

# 1) Cargar el modelo entrenado
clf = joblib.load('models/rf_classifier.pkl')

# 2) Definir columnas de features (incluyendo las nuevas)
feature_cols = [
    'latitud','longitud','categoria_negocio',
    'dist_city_center_km','local_density_1km','cluster_zone'
]

# 3) Predecir clase y probabilidad máxima
df['pred_clase'] = clf.predict(df[feature_cols])
df['pred_proba'] = clf.predict_proba(df[feature_cols]).max(axis=1)

# 4) Crear mapa base centrado en city_center
m_pred = folium.Map(location=city_center, zoom_start=12)

# 5) Añadir marcadores coloreados por predicción
colors = {'baja':'red','media':'orange','alta':'green'}
for _, r in df.iterrows():
    folium.CircleMarker(
        location=(r.latitud, r.longitud),
        radius=4,
        color=colors.get(r.pred_clase, 'blue'),
        fill=True,
        fill_color=colors.get(r.pred_clase, 'blue'),
        fill_opacity=0.7,
        popup=(
            f"ID: {r.get('id_local','-')}<br>"
            f"Predicción: {r.pred_clase} ({r.pred_proba:.2f})<br>"
            f"Categoría: {r.categoria_negocio}"
        )
    ).add_to(m_pred)

# 6) Mostrar mapa con predicciones
m_pred